In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

2025-03-05 19:38:03.067235: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 19:38:04.634484: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 19:38:07.194454: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-05 19:38:19.740232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [19]:
# Load dataset (assuming it's in a CSV file)
df = pd.read_csv("./spam_ham_dataset.csv")  # Update the filename

# Check dataset structure
print(df.head())

# Convert labels: 'ham' -> 0, 'spam' -> 1
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})



  label                                               text  label_num
0   ham  Subject: enron methanol ; meter # : 988291\nth...          0
1   ham  Subject: hpl nom for january 9 , 2001\n( see a...          0
2   ham  Subject: neon retreat\nho ho ho , we ' re arou...          0
3  spam  Subject: photoshop , windows , office . cheap ...          1
4   ham  Subject: re : indian springs\nthis deal is to ...          0


In [20]:
# Split dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label_num'], test_size=0.2, random_state=42)

# Tokenization
max_vocab = 5000  # Max number of words
max_length = 100  # Max email length

tokenizer = Tokenizer(num_words=max_vocab, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Padding sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post")

In [22]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_vocab, output_dim=16, input_length=max_length),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification (Spam vs. Not Spam)
])

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

# Train model
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), batch_size=32)

model.save('./saved_model/Q_5.keras')

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 14s 55ms/step - accuracy: 0.7743 - loss: 0.5223 - val_accuracy: 0.9304 - val_loss: 0.1982
Epoch 2/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.9362 - loss: 0.1802 - val_accuracy: 0.9546 - val_loss: 0.1542
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.9704 - loss: 0.1061 - val_accuracy: 0.9604 - val_loss: 0.1290
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.9591 - loss: 0.1389 - val_accuracy: 0.9681 - val_loss: 0.1118
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.9862 - loss: 0.0612 - val_accuracy: 0.9652 - val_loss: 0.1269
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.9898 - loss: 0.0424 - val_accuracy: 0.9720 - val_loss: 0.1069
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.9955 - loss: 0.0254 - val_accuracy: 0.9710 - val_loss: 0.1082
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.9896 - loss: 0.0493 - val_acc

In [33]:
# Evaluate on test set

model = tf.keras.models.load_model('./saved_model/Q_5.keras')
test_loss, test_acc = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {test_acc:.4f}")


33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9741 - loss: 0.1169
Test Accuracy: 0.9758


In [35]:
# Function to classify emails
def classify_email(email_text):
    sequence = tokenizer.texts_to_sequences([email_text])
    padded = pad_sequences(sequence, maxlen=max_length, padding="post")
    prediction = model.predict(padded)[0, 0]
  
    return "Spam" if prediction > 0.5 else "Not Spam"

# Example test email
# sample_email = "Congratulations! You've won a $1000 gift card. Click here to claim."
# sample_email = "text 3	Subject: neon retreat ho ho ho , we ' re around to that most wonderful time of the year - - - neon leaders retreat time ! i know that this time of year is extremely hectic , and that it ' s tough to think about anything past the holidays , but life does go on past the week of december 25 through january 1 , and that ' s what i ' d like you to think about for a minute . on the calender that i handed out at the beginning of the fall semester , the retreat was scheduled for the weekend of january 5 - 6 . but because of a youth ministers conference that brad and dustin are connected with that week , we ' re going to change the date to the following weekend , january 12 - 13 . now comes the part you need to think about . i think we all agree that it ' s important for us to get together and have some time to recharge our batteries before we get to far into the spring semester , but it can be a lot of trouble and difficult for us to get away without kids , etc . so , brad came up with a potential alternative for how we can get together on that weekend , and then you can let me know which you prefer . the first option would be to have a retreat similar to what we ' ve done the past several years . this year we could go to the heartland country inn ( www . . com ) outside of brenham . it ' s a nice place , where we ' d have a 13 - bedroom and a 5 - bedroom house side by side . it ' s in the country , real relaxing , but also close to brenham and only about one hour and 15 minutes from here . we can golf , shop in the antique and craft stores in brenham , eat dinner together at the ranch , and spend time with each other . we ' d meet on saturday , and then return on sunday morning , just like what we ' ve done in the past . the second option would be to stay here in houston , have dinner together at a nice restaurant , and then have dessert and a time for visiting and recharging at one of our homes on that saturday evening . this might be easier , but the trade off would be that we wouldn ' t have as much time together . i ' ll let you decide . email me back with what would be your preference , and of course if you ' re available on that weekend . the democratic process will prevail - - majority vote will rule ! let me hear from you as soon as possible , preferably by the end of the weekend . and if the vote doesn ' t go your way , no complaining allowed ( like i tend to do ! ) have a great weekend , great golf , great fishing , great shopping , or whatever makes you happy ! bobby"

sample_email = "I have important meeting today, So I will be late for the dinner"
print(f"Email: {sample_email}")
print(f"Prediction: {classify_email(sample_email)}")


Email: I have important meeting today, So Iwill be late for the dinner
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Prediction: Not Spam
